In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime
import glob

# Reading in data

In [ ]:
y = pd.read_excel('data/Returns/Ret_Var.xlsx', index_col=0)

In [ ]:
names = []
predictors = []

In [ ]:
files = glob.glob("data/HighTurnover/*.xlsx")
for file in files:
    predictors.append(pd.read_excel(file, index_col=0))
    names.append(file.rsplit('/')[-1].rsplit('.')[0])

In [ ]:
files = glob.glob("data/MidTurnover/*.xlsx")
for file in files:
    predictors.append(pd.read_excel(file, index_col=0))
    names.append(file.rsplit('/')[-1].rsplit('.')[0])

In [ ]:
files = glob.glob("data/LowTurnover/*.xlsx")
for file in files:
    predictors.append(pd.read_excel(file, index_col=0))
    names.append(file.rsplit('/')[-1].rsplit('.')[0])

In [ ]:
files = glob.glob("data/ChenCRSP/*.xlsx")
for file in files:
    predictors.append(pd.read_excel(file, index_col=0))
    names.append(file.rsplit('/')[-1].rsplit('.')[0])

In [ ]:
files = glob.glob("data/GHZ/*.xlsx")
for file in files:
    predictors.append(pd.read_excel(file, index_col=0))
    names.append(file.rsplit('/')[-1].rsplit('.')[0])

# Cleaning and preparing

In [ ]:
predictors_stacked_bfill = []
for i in range(len(predictors)):
    df_filled = predictors[i].fillna(method='ffill', axis=1)
    df_filled = df_filled.fillna(0)
    df_filled[pd.isna(y)] = np.NaN
    df_stack = df_filled.stack().dropna()
    df_stack.name = names[i]
    df_stack = df_stack.reset_index()
    predictors_stacked_bfill.append(df_stack)

In [ ]:
res_bfill = predictors_stacked_bfill[0]
for i in range(1,len(predictors)):
    predictors_stacked_bfill[i].rename(columns={predictors_stacked_bfill[i].columns[0]: "Permno"}, inplace=True)
    res_bfill = pd.merge(res_bfill, predictors_stacked_bfill[i], how="left", left_on=["Permno", "level_1"], right_on=["Permno", "level_1"])
res_bfill.iloc[:,2:] = res_bfill.iloc[:,2:].fillna(method='ffill', axis=1)
res_bfill.iloc[:,2:] = res_bfill.iloc[:,2:].T.fillna(res_bfill.iloc[:,2:].median(axis=1)).T
    

In [ ]:
res_bfill.shape

In [ ]:
res_bfill.to_csv("full_predictor_set_bfill.csv")

In [ ]:
y_stack = y.stack().dropna()
y_stack.name = "return"
y_stack = y_stack.reset_index()
y_stack.head()

In [ ]:
y_stack.to_csv("returns.csv")

# Fitting Models

In [ ]:
X = pd.read_csv("full_predictor_set_bfill.csv", index_col=0)
y = pd.read_csv("returns.csv", index_col=0)

In [ ]:
MSEs_ols, MSEs_ridge, MSEs_elastic = ([], [], [])
R2s_ols, R2s_ridge, R2s_elastic = ([], [], [])
i = 200000
while i + 1000 < 201900:
    x_train = X[(X["level_1"] >= i) & (X["level_1"] < i + 1000)].to_numpy()[:,2:]
    y_train = y[(y["level_1"] >= i) & (y["level_1"] < i + 1000)].to_numpy()[:,2:]
    x_test = X[(X["level_1"] >= i + 1000) & (X["level_1"] < i + 1100)].to_numpy()[:,2:]
    y_test = y[(y["level_1"] >= i + 1000) & (y["level_1"] < i + 1100)].to_numpy()[:,2:]
    
    # OLS
    reg = LinearRegression().fit(x_train, y_train)
    MSEs_ols.append(mean_squared_error(y_test, reg.predict(x_test)))
    R2s_ols.append(1-(np.sum((reg.predict(x_test)-y_test)**2)/np.sum(y_test**2)))
    
    i = i + 100
    
    # Ridge
    reg = Ridge().fit(x_train, y_train)
    MSEs_ridge.append(mean_squared_error(y_test, reg.predict(x_test)))
    R2s_ridge.append(1-(np.sum((reg.predict(x_test)-y_test)**2)/np.sum(y_test**2)))
    
    # Elastic Net
    reg = ElasticNet().fit(x_train, y_train)
    MSEs_elastic.append(mean_squared_error(y_test, reg.predict(x_test)))
    #R2s_elastic.append(1-(np.sum((reg.predict(x_test)-y_test)**2)/np.sum(y_test**2)))

In [ ]:
np.savetxt("MSEs_ols.csv", np.array(MSEs_ols), delimiter=",")
np.savetxt("MSEs_ridge.csv", np.array(MSEs_ridge), delimiter=",")
np.savetxt("MSEs_elastic.csv", np.array(MSEs_elastic), delimiter=",")
np.savetxt("R2s_ols.csv", np.array(R2s_ols), delimiter=",")
np.savetxt("R2s_ridge.csv", np.array(R2s_ridge), delimiter=",")
np.savetxt("R2s_elastic.csv", np.array(R2s_elastic), delimiter=",")

In [ ]:
plt.subplot(2,1,1)
plt.plot(MSEs_ols)
plt.title("MSEs")

plt.subplot(2,1,2)
plt.plot(R2s_ols)
plt.title("R2s")

plt.tight_layout()

In [ ]:
print(np.mean(MSEs_ols))
print(np.mean(R2s_ols))

Last MSE and R^2 value with mean filling
0.021918890935935076
-0.010627932052457923